In [1]:
import numpy as np
from scipy import stats as st

In [7]:
teta = 10
n = 100

In [8]:
data = teta * (np.random.rand(n) + 1)
data

array([17.84371725, 13.58917982, 17.60346578, 18.58795642, 19.24747901,
       13.16614698, 13.872367  , 10.62794169, 17.4248122 , 15.0082428 ,
       12.58853599, 10.12126728, 15.04009972, 15.91969297, 12.71054673,
       18.33897187, 17.1139887 , 12.18280997, 12.02930956, 13.70501971,
       13.94882276, 13.9568201 , 13.22783076, 14.25300831, 18.3930961 ,
       10.47949937, 10.2766708 , 17.0195793 , 15.98581225, 19.12666477,
       14.50779757, 13.45542374, 19.91782426, 17.42032831, 14.23754293,
       17.1305333 , 12.61478276, 11.3083398 , 10.95940051, 14.81012685,
       16.97660222, 12.54487204, 13.03197522, 10.46704494, 12.00823508,
       18.32231208, 13.09037223, 17.71745603, 11.20091676, 11.1072868 ,
       11.01145756, 10.65570393, 17.12471422, 16.3985633 , 17.05120151,
       19.82543881, 17.81681708, 10.21755728, 13.05814214, 13.63719354,
       16.10260714, 12.68568161, 16.34654539, 17.40254635, 11.6413865 ,
       12.07169376, 15.62928352, 11.97504083, 14.11599143, 11.49

In [10]:
assessment_teta = (len(data) + 1) / (2 * len(data) + 1) * np.max(data)
func = lambda x: (n + 1) / (2 * n + 1) * np.max(x) - assessment_teta
non_parametric_bootstrap_ci = st.bootstrap(
    (data, ), 
    func, 
    n_resamples=1_000, 
    method='basic',
    confidence_level=0.95
).confidence_interval

non_parametric_bootstrap_ci = (assessment_teta - non_parametric_bootstrap_ci.high, assessment_teta - non_parametric_bootstrap_ci.low)

In [12]:
from collections.abc import Callable
from typing import NamedTuple


class ConfidenceInterval(NamedTuple):
    low: float
    high: float


def parametric_bootstrap(
    data: np.ndarray,
    statistic: Callable[[np.ndarray], float],
    prob_model: Callable[[int], np.ndarray],
    n_resamples: int = 10_000,
    confidence_level: float = 0.95,
) -> ConfidenceInterval:
    n = len(data)
    bootstrap_data = np.ndarray((n_resamples, ), dtype=np.float32)

    for i in range(n_resamples):
        resample = prob_model(n)
        bootstrap_data[i] = statistic(resample)
    
    bootstrap_data.sort()

    return ConfidenceInterval(
        low=bootstrap_data[round((1 - confidence_level) / 2 * n_resamples) - 1],
        high=bootstrap_data[round((1 + confidence_level) / 2 * n_resamples) - 1]
    )


parametric_bootstrap_ci = parametric_bootstrap(
    data,
    func,
    lambda n: np.random.uniform(assessment_teta, assessment_teta * 2, n),
    n_resamples=50_000, 
    confidence_level=0.95
)

parametric_bootstrap_ci = (assessment_teta - parametric_bootstrap_ci.high, assessment_teta - parametric_bootstrap_ci.low)

In [14]:
exact_ci = (
    np.max(data) / (1 + np.power(0.975, 1 / n)),
    np.max(data) / (1 + np.power(0.025, 1 / n))
)

mean = np.mean(data)
deviation = np.std(data)

asymptotic_ci = (
    -1.96 * np.sqrt(4 / (n * 9)) * deviation + 2 / 3 * mean,
    1.96 * np.sqrt(4 / (n * 9)) * deviation + 2 / 3 * mean
)

print(
    f"{exact_ci = }",
    f"{asymptotic_ci = }",
    f"{non_parametric_bootstrap_ci = }",
    f"{parametric_bootstrap_ci = }",
    sep="\n\n"
)

exact_ci = (9.960172819506385, 10.142577435187922)
asymptotic_ci = (9.451382045862976, 10.185308308315632)
non_parametric_bootstrap_ci = (9.882731763139146, 10.008458956907214)
parametric_bootstrap_ci = (9.95992072654766, 10.141406178305568)
